In [ ]:
%load_ext autoreload
%autoreload 2

# Test smina (and AmberTools)

Some code is deactivated.<br/>
Now smina is run through by a function

### Needs rdkit>=2020.03.3

In [ ]:
import rdkit
assert '2020.03.3' <= rdkit.__version__

In [ ]:
from cbiprep.ligand_expo import LigandExpo
from cbiprep.smina import RunSmina, CheckRMSD
from cbiprep.jupyter_utils import draw_mol, check_dir
import pandas as pd
import os, time, sys, random

In [ ]:
lig_expo = LigandExpo()

In [ ]:
df = pd.read_pickle('index_2019.pkl.gz')

### SMINA docking as a function

Inputs:
- pdb_code
- ligand_name
- workdir = 'work'
- ncpu = os.cpu_count()
- num_modes = 4
- seed = 0

### SMINA for all PDBs

In [ ]:
DF = df[(df['type'] == 'Kd') & (df['lig_ok'] == True) & (df['refined'] == True)]

In [ ]:
indices = list(DF.index)
random.shuffle(indices)
for idx in indices:
    r = DF.loc[idx]
    pdb_code = r['pdb']
    ligand_name = r['lig']
    rmsd_oname = f'work/{pdb_code}/rmsd'
    if os.path.exists(rmsd_oname):
        continue
    print('-'*100)
    start = time.time()
    try:
        docked_sdf = RunSmina(pdb_code, ligand_name, ncpu=12)
    except:
        print(f'Error: {pdb_code} {ligand_name}', file=sys.stderr)
        continue
    with open(rmsd_oname, 'wt') as out:
        for i, rmsd in enumerate(CheckRMSD(pdb_code, ligand_name, docked_sdf), start=1):
            print(i, rmsd)
            print(i, rmsd, file=out)
    lapse = time.time() - start
    print(f'Lapse = {lapse:.1f}s')